Idea:

Assume only few spoofing attacks

Synchronize clocks assuming all broadcast locations are correct

Estimate positions for messages based on sensor timestamps

Remove messages with high deviations between broadcast locations and estimated locations

Re-synchronize clocks and re-check broadcast locations

In [18]:
import pandas as pd
import numpy as np
import os
import re
import position_estimator
from position_estimator import GeoPoint
from collections import defaultdict
from tqdm import tqdm
import itertools

DATA_LOCATION = "D:/thesis_data/1hour_complete_22_01_10_05/"
#DATA_LOCATION = "../raw_data/1hour/"

sensors = list()
sensors_rev = dict()
received = defaultdict(lambda: { "sensors": list() })

for file in tqdm(os.listdir(DATA_LOCATION)):
    assert re.match(r"^part-\d{5}$", file)
    with open(os.path.join(DATA_LOCATION, file), "r") as f:
        line = f.readline()
        if not line:
            continue
        assert line == "sensorType,sensorLatitude,sensorLongitude,sensorAltitude,timeAtServer,timeAtSensor,timestamp,rawMessage,sensorSerialNumber,RSSIPacket,RSSIPreamble,SNR,confidence\n"
        while (line := f.readline().strip()):
            sensorType,sensorLatitude,sensorLongitude,sensorAltitude,timeAtServer,timeAtSensor,timestamp,rawMessage,sensorSerialNumber,RSSIPacket,RSSIPreamble,SNR,confidence = line.split(',')
            
            if not position_estimator.is_relevant(rawMessage):
                continue

            if 'null' in [sensorLatitude, sensorLongitude, sensorAltitude, timeAtSensor]:
                continue
            
            sensorLatitude = float(sensorLatitude)
            sensorLongitude = float(sensorLongitude)
            sensorAltitude = float(sensorAltitude)
            timeAtSensor = float(timeAtSensor)

            # limit to europe for now
            if not 35 < sensorLatitude < 75:
                continue
            elif not -10 < sensorLongitude < 40:
                continue

            if not (sensorLatitude and sensorLongitude):
                continue

            if not "pos" in received[rawMessage]:
                try:
                    received[rawMessage]["pos"] = position_estimator.get_announced_pos(rawMessage, timeAtSensor)
                    assert -90 <= received[rawMessage]["pos"].lat <= 90
                    assert -180 <= received[rawMessage]["pos"].lon <= 180
                except:
                    #print("Couldn't get position :(")
                    del received[rawMessage]
                    continue

            sensor = (GeoPoint(sensorLatitude, sensorLongitude, sensorAltitude), sensorType)
            if sensor not in sensors_rev:
                sensors_rev[sensor] = len(sensors)
                sensors.append(sensor)

            if (timeAtSensor, sensors_rev[sensor]) not in received[rawMessage]["sensors"]:
                received[rawMessage]["sensors"].append((timeAtSensor, sensors_rev[sensor]))

    print("Sensors:", len(sensors))
    print("Received Messages:", len(received))
    if len(received) >= 200000:
        break # memory constraint



  0%|          | 1/200 [00:12<40:39, 12.26s/it]

Sensors: 725
Received Messages: 15954


  1%|          | 2/200 [00:27<45:30, 13.79s/it]

Sensors: 743
Received Messages: 34803


  2%|▏         | 3/200 [00:39<42:27, 12.93s/it]

...

Sensors: 867
Received Messages: 474774


 28%|██▊       | 57/200 [06:17<29:39, 12.44s/it]

Sensors: 868
Received Messages: 491579


 28%|██▊       | 57/200 [06:34<16:30,  6.93s/it]

Sensors: 869
Received Messages: 511717


In [19]:
# remove messages that have been sent more than once
print("Messages before:", len(received))
for msg in list(received.keys()):
    if len(set([s for t, s in received[msg]["sensors"]])) < len(received[msg]["sensors"]):
        del received[msg]
print("Messages after: ", len(received))

Messages before: 511717
Messages after:  478333


In [20]:
from position_estimator import GeoPoint
import traceback
C = 299792458 # light speed, meters per second

time_delta = [ [ [] for j in range(len(sensors)) ] for i in range(len(sensors)) ]

for msg in tqdm(received):
    if not received[msg]["pos"]:
        continue

    for t, s in list(received[msg]["sensors"]):
        dists = sorted([sensors[s][0].dist(sensors[e[1]][0]) for e in received[msg]["sensors"] if e[1] != s])
        if dists[int(len(dists)/10)] > 1e6: # ge 1000 km
            print("removing:", t, s, dists)
            received[msg]["sensors"].remove((t, s))

    try:
        sensor_dists_to_msg_origin = {x[1]: received[msg]["pos"].dist(sensors[x[1]][0]) for x in received[msg]["sensors"]}
        time_to_sensor = { s: x / C for s, x in sensor_dists_to_msg_origin.items() }
    except:
        traceback.print_exc()
        print(received[msg]["pos"].lat, received[msg]["pos"].lon, received[msg]["pos"].alt)
        print({x[1]: (sensors[x[1]][0].lat, sensors[x[1]][0].lon, sensors[x[1]][0].alt) for x in received[msg]["sensors"]})

    for (t1, s1), (t2, s2) in itertools.combinations(received[msg]["sensors"], 2):
        assert s1 != s2
        td = (t1 - time_to_sensor[s1]) - (t2 - time_to_sensor[s2])
        time_delta[s1][s2].append(td)
        time_delta[s2][s1].append(-td)

lens = []
for i in range(len(sensors)):
    for j in range(i+1, len(sensors)):
        lens.append(len(time_delta[i][j]))

np.histogram(lens)

  0%|          | 0/478333 [00:00<?, ?it/s]C:\Users\Lukas Baege\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\geopy\point.py:472: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  return cls(*args)
Traceback (most recent call last):
  File "C:\Users\LUKASB~1\AppData\Local\Temp/ipykernel_39212/267622478.py", line 18, in <module>
    sensor_dists_to_msg_origin = {x[1]: received[msg]["pos"].dist(sensors[x[1]][0]) for x in received[msg]["sensors"]}
  File "C:\Users\LUKASB~1\AppData\Local\Temp/ipykernel_39212/267622478.py", line 18, in <dictcomp>
    sensor_dists_to_msg_origin = {x[1]: received[msg]["pos"].dist(sensors[x[1]][0]) for x in received[m

149.80161 114.18915 8534.4
{4: (51.8297, 4.7637, 10.0), 9: (52.07604, 5.22358, 11.0), 20: (51.126764, 6.842491, 8.0), 38: (53.24312, 5.99151, 9.5), 47: (52.38283, 5.259066, 8.0), 58: (51.4137, 5.4335, 21.0), 57: (51.4414, 5.5226, 30.0), 64: (51.391, 5.438, 27.0), 69: (51.728, 3.7549, 10.0), 85: (52.592428, 7.286886, 43.81), 177: (52.1524, 4.6689, 11.0), 183: (53.26, 9.7187, 61.0), 194: (52.52035, 7.29957, 26.4), 196: (51.04332, 5.16706, 43.0), 95: (52.5772, 8.1373, 51.0), 154: (52.2602, 6.0739, 8.0), 238: (52.043823, 4.646932, -4.0), 336: (52.284961, 8.025576, 89.0), 346: (51.558435, 6.7351, 40.0), 406: (52.6549, 4.813, 4.8768), 415: (51.97143, 6.71426, 50.0), 463: (52.34606, 4.67771, 20.0), 468: (52.381, 4.844, 30.0), 485: (52.05064, 5.64182, 20.0)}


KeyError: 4

We now have all time deltas between stations that picked up the same signal.
We model the clock shifts using gaussian distributions, with some mean mu and standard deviation sigma

Then, to get time delta probability distribution between two stations that didn't measure the same message, we convolute the probability density functions of stations with directly known time-delta

Example:
sensors A and B measured the same 100 messages, so we can directly model their time delta distribution function as: td_AB = Gauss(mu_AB, sigma_AB)
Similarly, assume we estimate the density function for sensors B and C as Gauss(mu_BC, sigma_BC)

Now, to get the density function for sensors A and C, we can convolute these two density functions:
density function of time delta A-C = Gauss(mu_AB, sigma_AB) * Gauss(mu_BC, sigma_BC) = Gauss(mu_AB + mu_BC, sqrt(sigma_AB^2 + sigma_BC^2))

We do this for all paths going from A to C, and sum up all PDFs to get the final PDF for the time delta between A and C.
For efficiency, we could just do a "shortest path", where the path lengths are the sum of variances.
This way, we just take the "best" path, with least uncertainty.

In [29]:
import statistics

time_delta_gaussians =  [ [ None for j in range(len(sensors)) ] for i in range(len(sensors)) ]

# initialize with directly known time deltas
for i in range(len(sensors)):
    for j in range(len(sensors)):
        if len(time_delta[i][j]) > 1:
            mean = statistics.mean(time_delta[i][j])
            var = statistics.variance(time_delta[i][j], xbar=mean)
            time_delta_gaussians[i][j] = (mean, var)

# floyd's algorithm
for k in tqdm(range(len(sensors))):
    for i in range(len(sensors)):
        for j in range(len(sensors)):
            if time_delta_gaussians[i][k] is not None and time_delta_gaussians[k][j] is not None:
                new_variance = time_delta_gaussians[i][k][1] + time_delta_gaussians[k][j][1]
                if time_delta_gaussians[i][j] is None or time_delta_gaussians[i][j][1] > new_variance:
                    new_mean = time_delta_gaussians[i][k][0] + time_delta_gaussians[k][j][0]
                    time_delta_gaussians[i][j] = (new_mean, new_variance)


100%|██████████| 1364/1364 [07:30<00:00,  3.03it/s]


In [36]:
variances = [ time_delta_gaussians[i][j][1] for i,j in itertools.combinations(range(len(sensors)), 2) if time_delta_gaussians[i][j] is not None ]
num_conns = [ len(time_delta[i][j]) for i,j in itertools.combinations(range(len(sensors)), 2) if len(time_delta[i][j]) > 0]

print("Mean Variance:", statistics.mean(variances))
print("Median variance:", statistics.median(variances))
print("Variance modes:", statistics.multimode(variances))

print("Mean nConnections:", statistics.mean(num_conns))
print("Median nConnections:", statistics.median(num_conns))
print("nConnections modes:", statistics.multimode(num_conns))

Mean Variance: 45.997385069043624
Median variance: 5.753407907944094e-10
Variance modes: [0.0]
Mean nConnections: 41.629132798085074
Median nConnections: 19.0
nConnections modes: [1]


Using those time deltas, we can now check the aircraft locations

In [89]:
import scipy.optimize

def estimate_position(sensor_timestamps, sensor_ids, time_delta_gaussians, sensors):
    assert len(sensor_timestamps) >= 4
    assert len(sensor_timestamps) == len(sensor_ids)

    # we want to find a position p, for which we want to minimize some objective function.
    # this objective function is subject to the time deltas,
    # and the distances from the estimated position and the sensor positions
    # also, we should consider the variances in the time delta distributions:
    # if some time delta probability distribution has very big variance,
    # we can't trust that sensor to the same degree as a sensor with a small time delta variance

    # attempt 1:
    def residual_error(p):
        p = position_estimator.GeoPoint(*p)
        #print("p:", p.lat, p.lon, p.alt)
        err = 0
        for i,j in itertools.combinations(range(len(sensor_ids)), 2):
            if time_delta_gaussians[sensor_ids[i]][sensor_ids[j]] is None:
                continue

            dist_i = p.dist(position_estimator.GeoPoint(*sensors[sensor_ids[i]][:3]))
            dist_j = p.dist(position_estimator.GeoPoint(*sensors[sensor_ids[j]][:3]))
            t_i = sensor_timestamps[i] - dist_i / C
            t_j = sensor_timestamps[j] - dist_j / C
            #print(sensor_ids[i], sensor_ids[j], t_i, t_j, time_delta_gaussians[sensor_ids[i]][sensor_ids[j]])
            e = (t_i - t_j - time_delta_gaussians[sensor_ids[i]][sensor_ids[j]][0]) ** 2 / time_delta_gaussians[sensor_ids[i]][sensor_ids[j]][1]
            err += e
        print(err)
        return err

    def residual_error2(p):
        p = position_estimator.GeoPoint(*p)
        err = 0
        true_t = sensor_timestamps[0] - p.dist(position_estimator.GeoPoint(*sensors[sensor_ids[0]][:3])) / C
        for i in range(1, len(sensor_ids)):
            t = sensor_timestamps[i] - p.dist(position_estimator.GeoPoint(*sensors[sensor_ids[i]][:3])) / C
            err += true_t - t - time_delta_gaussians[sensor_ids[0]][sensor_ids[i]][0]
        #print(err, len(sensor_ids), err/len(sensor_ids))
        return err

    def residual_error3(p):
        p = position_estimator.GeoPoint(*p)
        err = 0
        for assumed_true in range(len(sensor_ids)):
            true_t = sensor_timestamps[sensor_ids[assumed_true]] - p.dist(position_estimator.GeoPoint(*sensors[sensor_ids[assumed_true]][:3])) / C
            for i in range(len(sensor_ids)):
                if i == assumed_true:
                    continue
                t = sensor_timestamps[i] - p.dist(position_estimator.GeoPoint(*sensors[sensor_ids[i]][:3])) / C
                err += true_t - t - time_delta_gaussians[sensor_ids[assumed_true]][sensor_ids[i]][0]
            #print(err, len(sensor_ids), err/len(sensor_ids))
            return err

    def residual_error4(p):
        p = position_estimator.GeoPoint(*p)
        # choose 4 best time_delta distributions (smallest variances)
        val = (100000, None)
        for a, b, c, d in itertools.combinations(range(len(sensor_ids)), 4):
            s = sum([time_delta_gaussians[sensor_ids[e]][sensor_ids[f]][1] for e,f in itertools.combinations([a,b,c,d],2)])
            val = min(val, (s, (a,b,c,d)))
        err = 0
        for i,j in itertools.combinations(val[1], 2):
            #print(i,j, sensor_ids[i], sensor_ids[j])
            ti = sensor_timestamps[i] - p.dist(position_estimator.GeoPoint(*sensors[sensor_ids[i]][:3])) / C
            tj = sensor_timestamps[j] - p.dist(position_estimator.GeoPoint(*sensors[sensor_ids[j]][:3])) / C
            err += ti - tj - time_delta_gaussians[sensor_ids[i]][sensor_ids[j]][0]
        return err


    bounds = scipy.optimize.Bounds([-90, -180, 0], [90, 180, 100000])
    method = 'L-BFGS-B'
    return position_estimator.GeoPoint(*scipy.optimize.minimize(residual_error4, sensors[sensor_ids[0]][:3], method=method, bounds=bounds).x)



for msg in tqdm(received):
    if len(received[msg]["sensors"]) < 4:
        continue
    
    estimated_pos = estimate_position(*zip(*received[msg]["sensors"]), time_delta_gaussians, sensors)
    received[msg]["estimated_pos"] = estimated_pos
    print("Broadcast pos:", received[msg]["pos"].lat, received[msg]["pos"].lon, received[msg]["pos"].alt)
    print("Estimated pos:", received[msg]["estimated_pos"].lat, received[msg]["estimated_pos"].lon, received[msg]["estimated_pos"].alt)
    print("Dist:", received[msg]["pos"].dist(received[msg]["estimated_pos"]))
    

  0%|          | 3/74616 [00:05<31:20:38,  1.51s/it]

Broadcast pos: 52.19307 5.83303 8534.4
Estimated pos: 52.35135451381608 4.6648244947577355 9.0
Dist: 82107.20090310114
Broadcast pos: 39.92888 -85.03836 11574.78
Estimated pos: 42.7024131101311 -79.95226419396874 207.0
Dist: 525554.8880443444


  0%|          | 4/74616 [00:05<22:12:46,  1.07s/it]

Broadcast pos: 39.95448 -87.92071 10972.800000000001
Estimated pos: 40.37118013138301 -86.90925484422935 207.0
Dist: 98393.37371406784
Broadcast pos: 41.16541 -87.06134 9753.6
Estimated pos: 40.38904124841154 -86.8769056189761 207.0
Dist: 88128.5383276204


  0%|          | 6/74616 [00:05<13:31:14,  1.53it/s]

...

  0%|          | 59/74616 [00:35<5:08:06,  4.03it/s]

Broadcast pos: 38.21813 -76.97326 11277.6
Estimated pos: 38.44451031711938 -77.90114465920067 0.0
Dist: 85677.87625414912


  0%|          | 60/74616 [00:35<12:11:54,  1.70it/s]

Broadcast pos: 51.12259 4.23709 1729.74
Estimated pos: 51.49782647215581 4.284365277610843 16.0
Dist: 41911.631854914216


TypeError: 'NoneType' object is not subscriptable